In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy import spatial
import os

In [2]:
def create_graph(edgelist_df):
    G = nx.Graph()
    for edge in list(zip(edgelist_df.user1_id, edgelist_df.user2_id)):
        G.add_edge(edge[0], edge[1])
    return G

def create_random_edges_dict(G, n):
    random_edges_dict = {}
    nodes_list = list(G.nodes)
    edges_list = list(G.edges)
    negative_samples_percentage = 0.001
    num_negative_samples = int(negative_samples_percentage*(len(nodes_list)*len(nodes_list)/2 - len(edges_list)))
    print("Number of negative samples in the random graph:" + str(num_negative_samples))
    for i in range(n):
        random_edges = []    
        while len(random_edges) < num_negative_samples:
            edge = random.sample(nodes_list, 2)
            if edge[0]==edge[1] or (edge[0], edge[1]) in random_edges or (edge[1], edge[0]) in random_edges or (edge[0], edge[1]) in edges_list or (edge[1], edge[0]) in edges_list:
                continue
            random_edges.append((edge[0], edge[1]))
        random_edges_dict[i] = random_edges
    return random_edges_dict

def calculate_relative_change_for_numeric_features(df, columns):
    for column in columns:
        df['relative_change_'+column] = df.apply(lambda x: relative_diff(x['user1_'+column], x['user2_'+column]), axis=1)
    return df

def calculate_cosine_similarity_for_vector_features(df, columns):
    for column in columns:
        df['cosine_similarity_'+column] = df.apply(lambda x: cosine_similarity(x['user1_'+column], x['user2_'+column]), axis=1)
    return df

def create_mask_for_categoric_features(df, columns):
    for column in columns:
        df['same_'+column] = (df['user1_'+column]==df['user2_'+column]).astype(int)
    return df

def calculate_jaccard_coefficient(df, G):
    df['jaccard_coefficient'] = df.apply(lambda x: get_jaccard_coefficient(G, x['user1_user_id'], x['user2_user_id']), axis=1)
    return df

def calculate_adamic_adar_coefficient(df, G):
    df['adamic_adar_coefficient'] = df.apply(lambda x: get_adamic_adar_index(G, x['user1_user_id'], x['user2_user_id']), axis=1)
    return df

def calculate_common_neighbors(df, G):
    df['common_neighbors'] = df.apply(lambda x: get_common_neighbors(G, x['user1_user_id'], x['user2_user_id']), axis=1)
    return df

def relative_diff(x, y):
    if x is None or y is None or x==-1 or y==-1:
        return None
    if max(x,y) == 0:
        return 0
    else:
        return abs((abs(x)-abs(y))/max(x,y))

def cosine_similarity(list1, list2):
    return 1 - spatial.distance.cosine(list1, list2)

def get_jaccard_coefficient(G, user1, user2):
    return list(nx.jaccard_coefficient(G, [(user1, user2)]))[0][2]

def get_adamic_adar_index(G, user1, user2):
    return list(nx.adamic_adar_index(G, [(user1, user2)]))[0][2]

def get_common_neighbors(G, user1, user2):
    return len(list(nx.common_neighbors(G, user1, user2)))

def extend_users_df_with_relational_features(users_df):
    users_df['freebase_nmf_array'] = users_df[['freebase_nmf_0', 'freebase_nmf_1', 'freebase_nmf_2', 'freebase_nmf_3', 'freebase_nmf_4', 'freebase_nmf_5', 'freebase_nmf_6', 'freebase_nmf_7', 'freebase_nmf_8', 'freebase_nmf_9', 'freebase_nmf_10', 'freebase_nmf_11', 'freebase_nmf_12', 'freebase_nmf_13', 'freebase_nmf_14', 'freebase_nmf_15', 'freebase_nmf_16', 'freebase_nmf_17', 'freebase_nmf_18', 'freebase_nmf_19']].values.tolist()
    users_df['allmusic_genre_array'] = users_df[['allmusic_rnb', 'allmusic_rap', 'allmusic_electronic', 'allmusic_rock', 'allmusic_new age', 'allmusic_classical', 'allmusic_reggae', 'allmusic_blues', 'allmusic_country', 'allmusic_world', 'allmusic_folk', 'allmusic_easy listening', 'allmusic_jazz', 'allmusic_vocal', "allmusic_children's", 'allmusic_punk', 'allmusic_alternative', 'allmusic_spoken word', 'allmusic_pop', 'allmusic_heavy metal']].values.tolist()
    users_df['UAM_nmf_array'] = users_df[['UAM_nmf_0', 'UAM_nmf_1', 'UAM_nmf_2', 'UAM_nmf_3', 'UAM_nmf_4', 'UAM_nmf_5', 'UAM_nmf_6', 'UAM_nmf_7', 'UAM_nmf_8', 'UAM_nmf_9', 'UAM_nmf_10', 'UAM_nmf_11', 'UAM_nmf_12', 'UAM_nmf_13', 'UAM_nmf_14', 'UAM_nmf_15', 'UAM_nmf_16', 'UAM_nmf_17', 'UAM_nmf_18', 'UAM_nmf_19']].values.tolist()
    users_df['allmusic_genre_array'] = users_df['allmusic_genre_array'].apply(lambda x: np.array(x))
    users_df['freebase_nmf_array'] = users_df['freebase_nmf_array'].apply(lambda x: np.array(x))
    users_df['UAM_nmf_array'] = users_df['UAM_nmf_array'].apply(lambda x: np.array(x))
    return users_df

def create_links_df(G, random_edges, users_df):
    edges_list = list(G.edges)
    users1_df = pd.DataFrame([x[0] for x in edges_list]).rename(columns={0:'user_id'}).merge(users_df, how='left', on='user_id').add_prefix('user1_')
    users2_df = pd.DataFrame([x[1] for x in edges_list]).rename(columns={0:'user_id'}).merge(users_df, how='left', on='user_id').add_prefix('user2_')
    friendship_df = pd.concat([users1_df, users2_df.set_index(users1_df.index)], axis=1)
    friendship_df['label'] = 1
    users1_df = pd.DataFrame([x[0] for x in random_edges]).rename(columns={0:'user_id'}).merge(users_df, how='left', on='user_id').add_prefix('user1_')
    users2_df = pd.DataFrame([x[1] for x in random_edges]).rename(columns={0:'user_id'}).merge(users_df, how='left', on='user_id').add_prefix('user2_')
    no_friendship_df = pd.concat([users1_df, users2_df.set_index(users1_df.index)], axis=1)
    no_friendship_df['label'] = 0
    links_df = pd.concat([friendship_df, no_friendship_df], axis=0, ignore_index=True)
    links_df = calculate_relative_change_for_numeric_features(links_df, ['age','playcount_lognorm','novelty_artist_avg_month','novelty_artist_avg_6months','novelty_artist_avg_year','mainstreaminess_avg_month','mainstreaminess_avg_6months','mainstreaminess_avg_year','mainstreaminess_global','relative_le_per_weekday1','relative_le_per_weekday2','relative_le_per_weekday3','relative_le_per_weekday4','relative_le_per_weekday5','relative_le_per_weekday6','relative_le_per_weekday7','relative_le_per_hour0','relative_le_per_hour1','relative_le_per_hour2','relative_le_per_hour3','relative_le_per_hour4','relative_le_per_hour5','relative_le_per_hour6','relative_le_per_hour7','relative_le_per_hour8','relative_le_per_hour9','relative_le_per_hour10','relative_le_per_hour11','relative_le_per_hour12','relative_le_per_hour13','relative_le_per_hour14','relative_le_per_hour15','relative_le_per_hour16','relative_le_per_hour17','relative_le_per_hour18','relative_le_per_hour19','relative_le_per_hour20','relative_le_per_hour21','relative_le_per_hour22','relative_le_per_hour23','cnt_listeningevents_lognorm','cnt_distinct_tracks_lognorm','cnt_distinct_artists_lognorm','cnt_listeningevents_per_week_lognorm','allmusic_weighted_average_diversity','allmusic_genre_coverage_diversity','allmusic_entropy_diversity','freebase_weighted_average_diversity','freebase_genre_coverage_diversity','freebase_entropy_diversity'])
    links_df = create_mask_for_categoric_features(links_df, ['country','gender','age_group','user_groups_freebase_weighted_average_diversity','user_groups_freebase_genre_coverage_diversity','user_groups_freebase_entropy_diversity','user_groups_allmusic_weighted_average_diversity','user_groups_allmusic_genre_coverage_diversity','user_groups_allmusic_entropy_diversity','user_groups_cnt_listeningevents_lognorm','user_groups_cnt_distinct_tracks_lognorm','user_groups_cnt_distinct_artists_lognorm','user_groups_cnt_listeningevents_per_week_lognorm','user_groups_playcount_lognorm','user_groups_novelty_artist_avg_month','user_groups_novelty_artist_avg_6months','user_groups_novelty_artist_avg_year','user_groups_mainstreaminess_avg_month','user_groups_mainstreaminess_avg_6months','user_groups_mainstreaminess_avg_year','user_groups_mainstreaminess_global'])
    links_df = calculate_cosine_similarity_for_vector_features(links_df, ['allmusic_genre_array', 'freebase_nmf_array', 'UAM_nmf_array'])
    links_df = links_df.select_dtypes(exclude=['object'])
    links_df = calculate_jaccard_coefficient(links_df, G)
    links_df = calculate_adamic_adar_coefficient(links_df, G)
    links_df = calculate_common_neighbors(links_df, G)
    return links_df

def create_and_store_dfs(G, random_edges_dict, users_df):
    for i in range(len(random_edges_dict)):
        links_df = create_links_df(G, random_edges_dict[i], users_df)
        outdir = '../data/dataframes/links_dfs/'+ str(i)
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        links_df.to_csv(outdir + '/links_df_full.csv')
        network_features_df = links_df[['user1_user_id', 'user2_user_id', 'jaccard_coefficient', 'adamic_adar_coefficient', 'common_neighbors', 'label']]
        demographic_features_df = pd.concat([links_df.filter(regex=("user1_user_id")), links_df.filter(regex=("user2_user_id")), links_df.filter(regex=(".*gender.*")), links_df.filter(regex=(".*age.*")), links_df.filter(regex=(".*country.*")), links_df.label], axis=1).drop(columns=['user1_allmusic_country', 'user2_allmusic_country'])
        demographic_features_df = demographic_features_df[demographic_features_df.columns.drop(list(demographic_features_df.filter(regex='allmusic')))]
        demographic_features_df = demographic_features_df[demographic_features_df.columns.drop(list(demographic_features_df.filter(regex='freebase')))]
        genre_features_df = pd.concat([links_df.filter(regex=("user1_user_id")), links_df.filter(regex=("user2_user_id")), links_df.filter(regex=(".*allmusic.*")), links_df.filter(regex=(".*freebase.*")), links_df.label], axis=1)
        genre_features_df = genre_features_df[genre_features_df.columns.drop(list(genre_features_df.filter(regex='diversity')))]
        listening_profile_features_df = pd.concat([links_df.filter(regex=("user1_user_id")), links_df.filter(regex=("user2_user_id")), links_df.filter(regex=(".*UAM.*")), links_df.label], axis=1)
        listening_characteristics_features_df = pd.concat([links_df.filter(regex=("user1_user_id")), links_df.filter(regex=("user2_user_id")), links_df.filter(regex=(".*mainstreaminess.*")), links_df.filter(regex=(".*novelty.*")), links_df.filter(regex=(".*diversity.*")), links_df.filter(regex=(".*cnt.*")), links_df.filter(regex=(".*playcount.*")), links_df.label], axis=1)
        network_features_df.to_csv('../data/dataframes/links_dfs/' + str(i) + '/links_df_network_features.csv')
        demographic_features_df.to_csv('../data/dataframes/links_dfs/' + str(i) + '/links_df_demographic_features.csv')
        genre_features_df.to_csv('../data/dataframes/links_dfs/' + str(i) + '/links_df_genre_features.csv')
        listening_profile_features_df.to_csv('../data/dataframes/links_dfs/' + str(i) + '/links_df_listening_profile_features.csv')
        listening_characteristics_features_df.to_csv('../data/dataframes/links_dfs/' + str(i) + '/links_df_listening_characteristics_features.csv')

In [3]:
lfm1b_user_info_filepath = '../data/raw/LFM-1b_social/LFM-1b_users.txt'
lfm1b_user_additional_info_filepath = '../data/raw/LFM-1b/LFM-1b_users_additional.txt'
lfm1b_user_genres_allmusic_filepath = '../data/raw/LFM-1b_UGP/LFM-1b_UGP_weightedPC_allmusic.txt'
lfm1b_user_genres_freebase_filepath = '../data/raw/LFM-1b_UGP/LFM-1b_UGP_weightedPC_freebase.txt'
input_edgelist_csv_filepath = '../data/raw/LFM-1b_social/LFM-1b_social_ties.txt'
lfm1b_user_artists_LEs_filepath = '../data/raw/LFM-1b/LFM-1b_LEs.mat'
users_df_filepath = ('../data/dataframes/users_dfs/users_df.csv')

In [4]:
lfm1b_users_df = pd.read_csv(lfm1b_user_info_filepath, sep='\t')
edgelist_df = pd.read_csv(input_edgelist_csv_filepath, sep='\t')
G = create_graph(edgelist_df)

num_datasets = 1
random_edges_dict = create_random_edges_dict(G, num_datasets)

users_df = pd.read_csv(users_df_filepath)
users_df = extend_users_df_with_relational_features(users_df)

users_df_without_missing_values = pd.read_csv('../data/dataframes/users_dfs/users_df_no_missing_values.csv', index_col=0)
users_df_without_missing_values = extend_users_df_with_relational_features(users_df_without_missing_values)

create_and_store_dfs(G, random_edges_dict, users_df)

Number of negative samples in the random graph:69446


In [ ]:
links_df = pd.read_csv('../data/dataframes/links_dfs/0/links_df_full.csv', index_col=False).drop(columns=['Unnamed: 0'])